In [ ]:
## importing the dependencies
# importing necessary dependencies.

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import obspy
import seaborn as sns
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.offsetbox import AnchoredText
import matplotlib as mpl
import tsfel

import matplotlib.cm as cm
from matplotlib.colors import Normalize  
from matplotlib import ticker


from scipy.interpolate import griddata as gd
import cartopy as cart
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import linkage
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

from cartopy.io.img_tiles import Stamen
import pygmt

from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram

In [ ]:
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['font.weight'] = 'bold'

In [ ]:
### Checking the folders present in the data directory
files = glob('Data/*')

file = glob('Data/*')[2]
vs = glob(file + '/*')

PO_HB = pd.read_csv(files[4], header = None, sep=' ')
X5 = pd.read_csv(files[5], header = None, sep=' ')

### storing station names and coordinates 
stations = []
for i in range(len(vs)):
    stations.append(vs[i].split('/')[2].split('_')[1])
    
stn_db = pd.concat([PO_HB,X5])
stn_db.index = stn_db[2]

stn_lons = []
stn_lats = []

for i in range(len(stations)):
    stn_lons.append(stn_db.at[stations[i],0])
    stn_lats.append(stn_db.at[stations[i],1])
    
features = []
vs_profiles = []
for i in tqdm(range(len(vs))):
    shear_velocity = pd.read_csv(vs[i], header=None, sep = ' ')[0]
    depth = pd.read_csv(vs[i], header=None, sep = ' ')[1]
    vs_profiles.append(shear_velocity.values)
    a  = shear_velocity.describe()
    features.append(a.values)
    
    
features = []
vs_profiles = []
for i in tqdm(range(len(vs))):
    shear_velocity = pd.read_csv(vs[i], header=None, sep = ' ')[0]
    vs_profiles.append(shear_velocity.values)
    a  = shear_velocity.describe()
    features.append(a.values)

In [ ]:
### Importing and Plotting the fault lines

file = glob('Manuscript/Faults/Fault1*')[0]
fault1 = pd.read_csv(file, header=None, sep='\t')
fault1_lons = fault1.values[:,0]
fault1_lats = fault1.values[:,1]


file = glob('Manuscript/Faults/Fault2*')[0]
fault1 = pd.read_csv(file, header=None, sep='\t')
fault2_lons = fault1.values[:,0]
fault2_lats = fault1.values[:,1]


file = glob('Manuscript/Faults/Fault3*')[0]
fault1 = pd.read_csv(file, header=None, sep='\t')
fault3_lons = fault1.values[:,0]
fault3_lats = fault1.values[:,1]


file = glob('Manuscript/Faults/Fault4*')[0]
fault1 = pd.read_csv(file, header=None, sep='\t')
fault4_lons = fault1.values[:,0]
fault4_lats = fault1.values[:,1]

file = glob('Manuscript/Faults/Fault5*')[0]
fault1 = pd.read_csv(file, header=None, sep='\t')
fault5_lons = fault1.values[:,0]
fault5_lats = fault1.values[:,1]


def plot_dendrogram(quantity=np.array(vs_profiles),method='ward', stn_labels=stations, d=[0,25]):

    X_primary = quantity[:,d[0]:d[1]]
    scaler = StandardScaler().fit(X_primary)
    X = scaler.transform(X_primary)
    df = df = pd.DataFrame(X,index=stn_labels)
    row_clusters = hierarchy.linkage(df.values,method=method,metric='euclidean')
    hierarchy.set_link_color_palette(['red','red','red','red'])
    plt.figure(figsize=[10,5], facecolor='white')
    row_dendr = hierarchy.dendrogram(row_clusters,labels=stn_labels)
    #plt.title(, fontsize=15)
    plt.ylabel("Distance",fontsize=12)
    plt.xticks(fontsize=10)
    #plt.ylim(0,30)
    plt.tight_layout()
    
def cluster_profiles(quantity=np.array(vs_profiles), n_clusters=4, linkage='ward', d=[0,25], cluster='ac'):
    ### Applying pre-processing
    X_primary = quantity[:,d[0]:d[1]]
    scaler = StandardScaler().fit(X_primary)
    X = scaler.transform(X_primary)

    if cluster == 'ac':
        clustering = AgglomerativeClustering(n_clusters=n_clusters,affinity='euclidean',
                                         linkage=linkage, compute_distances=True).fit(np.array(X))
        label = clustering.labels_
        
    if cluster == 'kmm':
        clustering = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
        label = clustering.labels_
        
    #label = clustering.labels_
    
    
    if cluster == 'gmm':
        label = GaussianMixture(n_components=n_clusters, random_state=0).fit_predict(X)
        
    return label

color = ['red', 'blue', 'green', 'orange','brown']
def plot_map(label, color=color, stn_lats = stn_lats, stn_lons=stn_lons, legend='legend1.txt'):
    
        ### PyGMT script
        region=[-105,-55, 50, 75]
        fig = pygmt.Figure()

        proj = 'S-80/90/6c'
        pygmt.makecpt(cmap='etopo1',series='-10000/10000/1000',continuous=True)

        #topo_data = '@earth_relief_01m'

        #fig.grdimage(grid=topo_data,region=region,projection='S-75/90/7i',cmap='geo',shading=True,frame='a')

        pygmt.config(FORMAT_GEO_MAP="ddd.x", MAP_FRAME_TYPE="plain",FONT_ANNOT_PRIMARY=7)

        fig.coast(area_thresh = 10000,
            projection=proj,
            region=region,
            shorelines=True,
            water="lightblue",
            resolution='i',
                 frame=["wsne"])

        fig.plot(x=fault1_lons,y=fault1_lats,style='c0.03c',color='black',pen='black',)
        fig.plot(x=fault2_lons,y=fault2_lats,style='c0.03c',color='black',pen='black',)
        fig.plot(x=fault3_lons,y=fault3_lats,style='c0.03c',color='black',pen='black',)
        fig.plot(x=fault4_lons,y=fault4_lats,style='c0.03c',color='black',pen='black',)
        fig.plot(x=fault5_lons,y=fault5_lats,style='c0.03c',color='black',pen='black',)
      



        for i in range(len(stn_lats)):
            fig.plot(
            x=stn_lons[i],
            y=stn_lats[i],
            style='c0.20c',
            color=color[int(label[i])],
            pen='black',
            )



        fig.text(projection='S-75/90/3i',
                text="0-50 Km, Clusters:2",
                position=[-80,75],
                font="12p,Helvetica-Bold,black",
                justify="TC",
            )

        #fig.text(text=stntexts,y=stnlats+0.01,x=stnlons+0.07,font="6p,Helvetica-Bold,blue")
        fig.legend(spec='Legends/'+legend,projection=proj, transparency=0, 
                   position = 'JTR+jTR+o-0.5c/-0.0c', 
                   box = '+gwhite+p1p')
        fig.show()
        return fig

In [ ]:
file = glob('Data3/*e/*')
file

In [ ]:
file_001_448 = glob(file[0]+'/*')
file_05_448 = glob(file[3]+'/*')
file_01_468 = glob(file[2]+'/*')
file_01_428 = glob(file[1]+'/*')
file_5_448 = glob(file[5]+'/*')
file_1_448 = glob(file[4]+'/*')


In [ ]:
print(len(file_001_448))
print(len(file_01_428))
print(len(file_01_468))
print(len(file_05_448))
print(len(file_1_448))
print(len(file_5_448))

In [ ]:
stations_001_448 = [] 
stations_05_448 = [] 
stations_01_468 = [] 
stations_01_428 = [] 
stations_5_448 = [] 
stations_1_448 = [] 


for i in range(len(file_001_448)):
    stations_001_448.append(file_001_448[i].split('/')[-1].split('_')[1])
    stations_05_448.append(file_05_448[i].split('/')[-1].split('_')[1])
    stations_1_448.append(file_1_448[i].split('/')[-1].split('_')[1])
    
for i in range(len(file_01_468)):
    stations_01_468.append(file_01_468[i].split('/')[-1].split('_')[1])
    stations_01_428.append(file_01_428[i].split('/')[-1].split('_')[1])
    
for i in range(len(file_5_448)): 
    stations_5_448.append(file_5_448[i].split('/')[-1].split('_')[1])
    
    
    
stations_001_448 = np.array(stations_001_448)
stations_01_468 = np.array(stations_01_468)
stations_01_428 = np.array(stations_01_428)
stations_05_448 = np.array(stations_05_448)
stations_1_448 = np.array(stations_1_448)
stations_5_448 = np.array(stations_5_448)

In [ ]:
Vs_001_448 = []
Vs_05_448 = []
Vs_01_468 = []
Vs_01_428 = []
Vs_5_448 = []
Vs_1_448  = []

for i in range(len(file_001_448)):
    df = pd.read_csv(file_001_448[i],skiprows=11,delimiter="   ")
    Vs_001_448.append(df.values[:,2])
    
    df = pd.read_csv(file_05_448[i],skiprows=11,delimiter="   ")
    Vs_05_448.append(df.values[:,2])
    
    df = pd.read_csv(file_1_448[i],skiprows=11,delimiter="   ")
    Vs_1_448.append(df.values[:,2])
    
for i in range(len(file_01_468)):  
    df = pd.read_csv(file_01_468[i],skiprows=11,delimiter="   ")
    Vs_01_468.append(df.values[:,2])
    
    df = pd.read_csv(file_01_428[i],skiprows=11,delimiter="   ")
    Vs_01_428.append(df.values[:,2])
    

for i in range(len(file_5_448)): 
    df = pd.read_csv(file_5_448[i],skiprows=11,delimiter="   ")
    Vs_5_448.append(df.values[:,2])
    
    
Vs_001_448 = np.array(Vs_001_448)
Vs_05_448 = np.array(Vs_05_448)

Vs_01_428 = np.array(Vs_01_428)
Vs_01_468 = np.array(Vs_01_468)

Vs_5_448 = np.array(Vs_5_448)
Vs_1_448  = np.array(Vs_1_448)

In [ ]:
stn_lons_001_448 = []
stn_lats_001_448 = []

for i in range(len(stations_001_448)):
    stn_lons_001_448.append(stn_db.at[stations_001_448[i],0])
    stn_lats_001_448.append(stn_db.at[stations_001_448[i],1])
stn_lons_001_448 = np.array(stn_lons_001_448)
stn_lats_001_448 = np.array(stn_lats_001_448)

stn_lons_05_448 = []
stn_lats_05_448 = []

for i in range(len(stations_05_448)):
    stn_lons_05_448.append(stn_db.at[stations_05_448[i],0])
    stn_lats_05_448.append(stn_db.at[stations_05_448[i],1])
stn_lons_05_448 = np.array(stn_lons_05_448)
stn_lats_05_448 = np.array(stn_lats_05_448)

stn_lons_01_468 = []
stn_lats_01_468 = []

for i in range(len(stations_01_468)):
    stn_lons_01_468.append(stn_db.at[stations_01_468[i],0])
    stn_lats_01_468.append(stn_db.at[stations_01_468[i],1])
stn_lons_01_468 = np.array(stn_lons_01_468)
stn_lats_01_468 = np.array(stn_lats_01_468)



stn_lons_01_428 = []
stn_lats_01_428 = []

for i in range(len(stations_01_428)):
    stn_lons_01_428.append(stn_db.at[stations_01_428[i],0])
    stn_lats_01_428.append(stn_db.at[stations_01_428[i],1])
stn_lons_01_428 = np.array(stn_lons_01_428)
stn_lats_01_428 = np.array(stn_lats_01_428)


stn_lons_5_448 = []
stn_lats_5_448 = []

for i in range(len(stations_5_448)):
    stn_lons_5_448.append(stn_db.at[stations_5_448[i],0])
    stn_lats_5_448.append(stn_db.at[stations_5_448[i],1])
stn_lons_5_448 = np.array(stn_lons_5_448)
stn_lats_5_448 = np.array(stn_lats_5_448)


stn_lons_1_448 = []
stn_lats_1_448 = []

for i in range(len(stations_1_448)):
    stn_lons_1_448.append(stn_db.at[stations_1_448[i],0])
    stn_lats_1_448.append(stn_db.at[stations_1_448[i],1])
stn_lons_1_448 = np.array(stn_lons_1_448)
stn_lats_1_448 = np.array(stn_lats_1_448)

In [ ]:
std_001_448 = []
for i in range(len(Vs_001_448)):
    df = pd.DataFrame(Vs_001_448[i].astype('float'))
    std_001_448.append(df.describe().values[2][0])
    
thresh_001_448 = np.percentile(np.array(std_001_448),90)

#####################
std_05_448 = []
for i in range(len(Vs_05_448)):
    df = pd.DataFrame(Vs_05_448[i].astype('float'))
    std_05_448.append(df.describe().values[2][0])
    
thresh_05_448 = np.percentile(np.array(std_05_448),90)

########################
std_01_468 = []
for i in range(len(Vs_01_468)):
    df = pd.DataFrame(Vs_01_468[i].astype('float'))
    std_01_468.append(df.describe().values[2][0])
    
thresh_01_468 = np.percentile(np.array(std_01_468),90)
########################


std_01_428 = []
for i in range(len(Vs_01_428)):
    df = pd.DataFrame(Vs_01_428[i].astype('float'))
    std_01_428.append(df.describe().values[2][0])
    
thresh_01_428 = np.percentile(np.array(std_01_428),90)


######################################


std_5_448 = []
for i in range(len(Vs_5_448)):
    df = pd.DataFrame(Vs_5_448[i].astype('float'))
    std_5_448.append(df.describe().values[2][0])
    
thresh_5_448 = np.percentile(np.array(std_5_448),90)


##########################################


std_1_448 = []
for i in range(len(Vs_1_448)):
    df = pd.DataFrame(Vs_1_448[i].astype('float'))
    std_1_448.append(df.describe().values[2][0])
    
thresh_1_448 = np.percentile(np.array(std_1_448),90)



In [ ]:
vs_001_448 = []
vlat_001_448 = []
vlon_001_448 = []
vstns_001_448 = []

plt.figure(figsize=[5,12])

for i in range(len(Vs_001_448)):
    if std_001_448[i] > thresh_001_448:
        plt.plot(Vs_001_448[i],depth,'-', color='k')
    else:
        plt.plot(Vs_001_448[i],depth,'-', color='orange')
        vs_001_448.append(Vs_001_448[i])
        vlat_001_448.append(stn_lats_001_448[i])
        vlon_001_448.append(stn_lons_001_448[i])
        vstns_001_448.append(stations_001_448[i])

plt.ylim(75,0) 
plt.xlim(2,6)
plt.ylabel('Depth [km]',fontsize=15)
plt.xlabel('Vs [km/s]',fontsize=15)
plt.title('p = 0.001, Vs = 4.48 km/s', fontweight='bold', fontsize=15)
plt.show()        
        
        
        
        
        
vs_05_448 = []
vlat_05_448 = []
vlon_05_448 = []
vstns_05_448 = []
plt.figure(figsize=[5,12])
for i in range(len(Vs_05_448)):
    if std_05_448[i] > thresh_05_448:
        plt.plot(Vs_05_448[i],depth,'-', color='k')
    else:
        plt.plot(Vs_05_448[i],depth,'-', color='orange')
        vs_05_448.append(Vs_05_448[i])
        vlat_05_448.append(stn_lats_05_448[i])
        vlon_05_448.append(stn_lons_05_448[i])
        vstns_05_448.append(stations_05_448[i])

plt.ylim(75,0) 
plt.xlim(2,6)
plt.ylabel('Depth [km]',fontsize=15)
plt.xlabel('Vs [km/s]',fontsize=15)
plt.title('p = 0.05, Vs = 4.48 km/s', fontweight='bold', fontsize=15)
plt.show()
        
        
        
        
        
vs_01_428 = []
vlat_01_428 = []
vlon_01_428 = []
vstns_01_428 = []
plt.figure(figsize=[5,12])
for i in range(len(Vs_01_428)):
    if std_01_428[i] > thresh_01_428:
        plt.plot(Vs_01_428[i],depth,'-', color='k')
    else:
        plt.plot(Vs_01_428[i],depth,'-', color='orange')
        vs_01_428.append(Vs_01_428[i])
        vlat_01_428.append(stn_lats_01_428[i])
        vlon_01_428.append(stn_lons_01_428[i])
        vstns_01_428.append(stations_01_428[i])


plt.ylim(75,0) 
plt.xlim(2,6)
plt.ylabel('Depth [km]',fontsize=15)
plt.xlabel('Vs [km/s]',fontsize=15)
plt.title('p = 0.01, Vs = 4.28 km/s', fontweight='bold', fontsize=15)
plt.show()


        
        
vs_01_468 = []
vlat_01_468 = []
vlon_01_468 = []
vstns_01_468 = []
plt.figure(figsize=[5,12])
for i in range(len(Vs_01_468)):
    if std_01_468[i] > thresh_01_468:
        plt.plot(Vs_01_468[i],depth,'-', color='k')
    else:
        plt.plot(Vs_01_468[i],depth,'-', color='orange')
        vs_01_468.append(Vs_01_468[i])
        vlat_01_468.append(stn_lats_01_468[i])
        vlon_01_468.append(stn_lons_01_468[i])
        vstns_01_468.append(stations_01_468[i])

plt.ylim(75,0) 
plt.xlim(2,6)
plt.ylabel('Depth [km]',fontsize=15)
plt.xlabel('Vs [km/s]',fontsize=15)
plt.title('p = 0.01, Vs = 4.68 km/s', fontweight='bold', fontsize=15)
plt.show()
        
        
        
        

vs_5_448 = []
vlat_5_448 = []
vlon_5_448 = []
vstns_5_448 = []
plt.figure(figsize=[5,12])
for i in range(len(Vs_5_448)):
    if std_5_448[i] > thresh_5_448:
        plt.plot(Vs_5_448[i],depth,'-', color='k')
    else:
        plt.plot(Vs_5_448[i],depth,'-', color='orange')
        vs_5_448.append(Vs_5_448[i])
        vlat_5_448.append(stn_lats_5_448[i])
        vlon_5_448.append(stn_lons_5_448[i])
        vstns_5_448.append(stations_5_448[i])        

plt.ylim(75,0) 
plt.xlim(2,6)
plt.ylabel('Depth [km]',fontsize=15)
plt.xlabel('Vs [km/s]',fontsize=15)
plt.title('p = 0.5, Vs = 4.48 km/s', fontweight='bold', fontsize=15)
plt.show()        
        
        
        
        

vs_1_448 = []
vlat_1_448 = []
vlon_1_448 = []
vstns_1_448 = []
plt.figure(figsize=[5,12])
for i in range(len(Vs_1_448)):
    if std_1_448[i] > thresh_1_448:
        plt.plot(Vs_1_448[i],depth,'-', color='k')
    else:
        plt.plot(Vs_1_448[i],depth,'-', color='orange')
        vs_1_448.append(Vs_1_448[i])
        vlat_1_448.append(stn_lats_1_448[i])
        vlon_1_448.append(stn_lons_1_448[i])
        vstns_1_448.append(stations_1_448[i]) 
        
        
plt.ylim(75,0) 
plt.xlim(2,6)
plt.ylabel('Depth [km]',fontsize=15)
plt.xlabel('Vs [km/s]',fontsize=15)
plt.title('p = 0.1, Vs = 4.48 km/s', fontweight='bold', fontsize=15)
plt.show()

In [ ]:
## 01, 448
plot_dendrogram(quantity=np.array(vs_profiles),method='ward', stn_labels=stations, d=[0,25])

In [ ]:
label = cluster_profiles(quantity=np.array(vs_profiles), n_clusters=5, linkage='ward', d=[0,25], cluster='ac')

In [ ]:
color = ['red', 'blue', 'green', 'orange','brown','yellow','cyan','violet','darkgreen']
plot_map(label=label, color=color, stn_lats = stn_lats, stn_lons=stn_lons, legend='legend5.txt')

In [ ]:
### 001, 448
plot_dendrogram(quantity=np.array(vs_001_448),method='ward', stn_labels=vstns_001_448, d=[0,25])

In [ ]:
label = cluster_profiles(quantity=np.array(vs_001_448), n_clusters=5, linkage='ward', d=[0,25], cluster='ac')

In [ ]:
color = ['red', 'blue', 'green', 'orange','brown','yellow','cyan','violet','darkgreen']
plot_map(label=label, color=color, stn_lats = vlat_001_448, stn_lons=vlon_001_448, legend='legend5.txt')

In [ ]:
### 05, 448
plot_dendrogram(quantity=np.array(vs_05_448),method='ward', stn_labels=vstns_05_448, d=[0,25])

In [ ]:
label = cluster_profiles(quantity=np.array(vs_05_448), n_clusters=5, linkage='ward', d=[0,25], cluster='ac')

In [ ]:
color = ['red', 'orange', 'blue', 'brown','green','yellow','cyan','violet','darkgreen']
plot_map(label=label, color=color, stn_lats = vlat_05_448, stn_lons=vlon_05_448, legend='legend5.txt')

In [ ]:
### 001, 448
plot_dendrogram(quantity=np.array(vs_01_428),method='ward', stn_labels=vstns_01_428, d=[0,25])

In [ ]:
label = cluster_profiles(quantity=np.array(vs_01_428), n_clusters=5, linkage='ward', d=[0,25], cluster='ac')

In [ ]:
color = ['red', 'green', 'blue', 'brown','orange','yellow','cyan','violet','darkgreen']
plot_map(label=label, color=color, stn_lats = vlat_01_428, stn_lons=vlon_01_428, legend='legend5.txt')

In [ ]:
### 01, 468
plot_dendrogram(quantity=np.array(vs_01_468),method='ward', stn_labels=vstns_01_468, d=[0,25])

In [ ]:
label = cluster_profiles(quantity=np.array(vs_01_468), n_clusters=5, linkage='ward', d=[0,25], cluster='ac')

In [ ]:
color = ['red', 'blue', 'green', 'orange','brown','yellow','cyan','violet','darkgreen']
plot_map(label=label, color=color, stn_lats = vlat_01_468, stn_lons=vlon_01_468, legend='legend5.txt')

In [ ]:
### 5, 448
plot_dendrogram(quantity=np.array(vs_5_448),method='ward', stn_labels=vstns_5_448, d=[0,25])

In [ ]:
label = cluster_profiles(quantity=np.array(vs_5_448), n_clusters=5, linkage='ward', d=[0,25], cluster='ac')

In [ ]:
color = ['red', 'orange', 'blue', 'brown','green','yellow','cyan','violet','darkgreen']
plot_map(label=label, color=color, stn_lats = vlat_5_448, stn_lons=vlon_5_448, legend='legend5.txt')

In [ ]:
### 1, 448
plot_dendrogram(quantity=np.array(vs_1_448),method='ward', stn_labels=vstns_1_448, d=[0,25])

In [ ]:
label = cluster_profiles(quantity=np.array(vs_1_448), n_clusters=5, linkage='ward', d=[0,25], cluster='ac')

In [ ]:
color = ['red', 'blue', 'green', 'brown','orange','yellow','cyan','violet','darkgreen']
plot_map(label=label, color=color, stn_lats = vlat_1_448, stn_lons=vlon_1_448, legend='legend5.txt')